# This Notebook is an attempt at experimenting with SVG file form. 

## Goal 1. Open an .svg file and extract the paths

## Goal 2. Convert the paths into meaningful points, tracking position after each command relative to the origin

## Goal 3. Create robot commands 

Before trying to run this make sure that you have Beautiful Soup and Python3 running in your interpreter

In [7]:
from bs4 import BeautifulSoup as bs # This is my personal favorite library for HTML and XML in Python
import re
#import numpy as np #might be useful latter
import math

# The next line opens file '76star.svg' of type 'xml'
soup = bs(open('2.png_SVG1.svg'), 'xml')
print(soup.prettify()) #bs.prettify prints a nicely formatted string, but soup is a tree

print('\n\n')
svg = soup.svg # get the svg info from soup

# finally we can find all of the path elements that are children of svg, and subsequently get the d elements iterating 
# through
for line in svg.find_all('line'): 
    print('line starts (',line.get('x1'), ',', line.get('y1'),') line ends (',line.get('x2'),',',line.get('y2'),')')
    

<?xml version="1.0" encoding="utf-8"?>
<svg baseProfile="full" height="541" version="1.1" width="773" xmlns="http://www.w3.org/2000/svg" xmlns:ev="http://www.w3.org/2001/xml-events" xmlns:xlink="http://www.w3.org/1999/xlink">
 <defs/>
 <g fill="none" id="shapes">
  <line stroke="rgb(10%,10%,16%)" x1="772" x2="495" y1="540" y2="20"/>
  <line stroke="rgb(10%,10%,16%)" x1="495" x2="542" y1="20" y2="28"/>
  <line stroke="rgb(10%,10%,16%)" x1="542" x2="554" y1="28" y2="36"/>
  <line stroke="rgb(10%,10%,16%)" x1="554" x2="571" y1="36" y2="52"/>
  <line stroke="rgb(10%,10%,16%)" x1="571" x2="579" y1="52" y2="64"/>
  <line stroke="rgb(10%,10%,16%)" x1="579" x2="587" y1="64" y2="84"/>
  <line stroke="rgb(10%,10%,16%)" x1="587" x2="578" y1="84" y2="155"/>
  <line stroke="rgb(10%,10%,16%)" x1="578" x2="561" y1="155" y2="177"/>
  <line stroke="rgb(10%,10%,16%)" x1="561" x2="645" y1="177" y2="185"/>
  <line stroke="rgb(10%,10%,16%)" x1="645" x2="670" y1="185" y2="206"/>
  <line stroke="rgb(10%,10%,


**Hopefully**, we successfully opened the '.svg' file and printed the data of the <line> tags. The first print statement should show the original file we worked with, while the second should show our targetted <line> data. It may not look like much, but this is nearly all we need. We also successuly retrieved each x,y pair point


In [37]:
def svg_to_lines(file_name):
    '''
    This method takes one parameter, an xml file_name. It extracts svg tags, and returns and list containing a tuple
    with a command and a list of points, which are also tuples.
    [(command, [(point1x, point1y), (point2x, point2y), ..., (pointNx, pointNy)]),]
    
    it's just lists of tuples all the way down
    '''
    #load file_name and get svg tag
    soup = bs(open(file_name), 'xml')
    svg = soup.svg
    
    lines = []
    #pat = re.compile(r'[.]+')
    for line in svg.find_all('line'):
        x1 = int(line.get('x1'))
        y1 = int(line.get('y1'))
        x2 = int(line.get('x2'))
        y2 = int(line.get('y2'))
        
        lines.append((x1,y1,x2,y2))
        
        
    return lines

example = svg_to_lines('2.png_SVG1.svg')
print('If our function works as expected, this output should be a list of 4-tuples')
for each in example:
    print(each)

If our function works as expected, this output should be a list of 4-tuples
(772, 540, 495, 20)
(495, 20, 542, 28)
(542, 28, 554, 36)
(554, 36, 571, 52)
(571, 52, 579, 64)
(579, 64, 587, 84)
(587, 84, 578, 155)
(578, 155, 561, 177)
(561, 177, 645, 185)
(645, 185, 670, 206)
(670, 206, 679, 219)
(679, 219, 687, 239)
(687, 239, 679, 304)
(679, 304, 670, 317)
(670, 317, 654, 334)
(654, 334, 642, 342)
(642, 342, 622, 350)
(622, 350, 550, 342)
(550, 342, 538, 334)
(538, 334, 521, 318)
(521, 318, 513, 306)
(513, 306, 505, 287)
(505, 287, 497, 299)
(497, 299, 489, 316)
(489, 316, 478, 330)
(478, 330, 468, 340)
(468, 340, 455, 348)
(455, 348, 386, 340)
(386, 340, 369, 324)
(369, 324, 361, 312)
(361, 312, 353, 293)
(353, 293, 361, 213)
(361, 213, 370, 199)
(370, 199, 384, 184)
(384, 184, 397, 176)
(397, 176, 212, 186)
(212, 186, 204, 230)
(204, 230, 187, 250)
(187, 250, 174, 258)
(174, 258, 106, 250)
(106, 250, 92, 237)
(92, 237, 84, 224)
(84, 224, 93, 155)
(93, 155, 104, 143)
(104, 143, 116, 13

## Goal #2
#### Convert the paths into meaningful points, tracking position after each command relative to the origin


In [12]:
'''
    Store location (x and y) and angle of robot. May add speed at some point
'''
class State:
    
    '''
        Initialize robot at Origin
        Origin is top left corner of paper with Robot pointing down. Down and Right are positive
        Why it's this way, I can't say
        SVG just liked it better that way
    '''
    def __init__(self, x=0.0, y=0.0, angle=0.0):
        self.x = x
        self.y = y
        self.angle = angle

## Goal #3
#### Create robot commands

In [32]:
''' Scratch Paper - figure your shit out'''

# determine angle and distance (duration * speed) e.i. cartesian coords to polar coords
# r = (x2 + y2)^1/2  
# θ = atan(y / x)  

# Getting that angle
point = (2 , 2)
state = State()
vector = {}

vector['x'] = point[0] - state.x
vector['y'] = point[1] - state.y
# determine angle and distance (duration * speed) e.i. cartesian coords to polar coords
# r = (x2 + y2)^1/2  
r = math.atan2(vector['y'], vector['x'])

print('arctan(', vector['y'], '/' , vector['x'] ,') = ', r , ' or ', math.degrees(r), ' degrees') 

def get_theta(x1, y1, x2, y2):
    '''
        takes four arguments x1, y1, x2, y2
        returns the angle in radians of arctan(y2-y1, x2-x1)
        
        Use this to convert from cartesian coordinates to polar coordinates
    '''
    r = math.atan2(y2-y1, x2-x1)
    return r

print('Our method says hopefully the same thing: ', get_theta(state.x, state.y, point[0], point[1]))

def get_distance(x1, y1, x2, y2):
    '''
        takes four arguments x1, y1, x2, y2
        returns the distance between the points (in no units or maybe all units)
        
        Use this to convert from cartesian coordinates to polar coordinates
    '''
    x, y = x2 - x1, y2-y1
    return(math.sqrt(math.pow(x, 2) + math.pow(y, 2)))
    
print('This should give the distance: ', get_distance(state.x, state.y, point[0], point[1]))

def cart_to_polar(x1, y1, x2, y2):
    '''
        takes four arguments x1, y1, x2, y2
        returns a dictionary with the angle in radians and distance between the points 
        {'dist': 2.8284271247461903, 'theta': 0.7853981633974483}
        
        Use this to convert from cartesian coordinates to polar coordinates
    '''
    return({'theta': get_theta(x1, y1, x2, y2), 'dist': get_distance(x1, y1, x2, y2)})
    
cart_to_polar(state.x, state.y, point[0], point[1])

arctan( 2.0 / 2.0 ) =  0.7853981633974483  or  45.0  degrees
Our method says hopefully the same thing:  0.7853981633974483
This should give the distance:  2.8284271247461903


{'dist': 2.8284271247461903, 'theta': 0.7853981633974483}

In [41]:


# Import the Robot.py file (must be in the same directory as this file!).

''' import Robot # UNCOMMENT THIS LINE ON ROBOT'''


LEFT_TRIM   = 0
RIGHT_TRIM  = -1


# Create an instance of the robot with the specified trim values.
# Not shown are other optional parameters:
#  - addr: The I2C address of the motor HAT, default is 0x60.
#  - left_id: The ID of the left motor, default is 1.
#  - right_id: The ID of the right motor, default is 2.
'''robot = Robot.Robot(left_trim=LEFT_TRIM, right_trim=RIGHT_TRIM) # UNCOMMENT THIS LINE ON ROBOT'''
FULL_REV = 4.85
DEF_SPEED = 50


def rotate(theta):
    ''' 
        this is going to be the place where we calibrate our robot, and we enter the real world, translating radians to
        1. direction of rotation (a simple if statement to turn the shortest way)
        2. speed of rotation (This should be fixed)
        3. duration of rotation 
    '''
    rev = FULL_REV # duration for one revolution or 2pi radians
    speed = DEF_SPEED # default speed
    
    #get smallest positive equivolent angle, which should never be needed
    while(theta > math.pi):
        theta = theta - math.pi*2
        
    if theta > math.pi: # rotate left
        #get equivolent negative angle
        dur = ((theta - 2*math.pi) - (2*math.pi)) * rev * -1 #keep time positive! rotate opposite
        '''robot.left(speed, dur) # UNCOMMENT THIS LINE ON ROBOT'''
    else: # rotate right
        dur = (theta / (2*math.pi)) * rev # 
        '''robot.right(speed, dur) # UNCOMMENT THIS LINE ON ROBOT'''
    
    
        

def go(distance):
    ''' 
        this is going to be the place where we calibrate our robot, and we enter the real world.
        Only two questions here. 
        1. How fast do we go?
        2. For how long?
    '''
    speed = DEF_SPEED
    dur = 12/1000 * distance
    '''robot.forward(speed, dur)# UNCOMMENT THIS LINE ON ROBOT'''
    
def go_to(state, points):
    '''
        Takes three arguments, state, points, and absolute=True
        point should be a list of 4-tuples representing x1,y1,x2,y2 respectively
    '''
    if type(points) is not tuple:
        print('improper argument type: ', type(points), ',', points)
        return
    if len(points) != 4:
        print('improper number of points in argument: ', len(points), 'found ,', points)
        return
    x1 = points[0]
    y1 = points[1]
    x2 = points[2]
    y2 = points[3]
    
    if x1 != state.x or y1 != state.y:
        print('state does not match command: (',state.x,',',state.y,') != (',x1,',',x2,') \ignorning, output may suffer' )
    
    # get vector in polar cordinates
    polar = cart_to_polar(x1, y1, x2, y2)
    # rotate angle
    rotate(polar['theta'])
    # go to there
    go(polar['dist'])
    
    # update and return? state, now at new point and angled in direction just travelled 
    state.x = x2 
    state.y = y2 
    state.angle =  polar['theta'] 
    return state
st = State()
print(go_to(st, (0,0,2,2)))

## I didn't think anyone would actually get this far

from here on out this is very much a work in progress and nothing is likely to work until I get further

Really the only part of this that is anywhere near complete is the Goal 1 portion, but it's a start

In [44]:
def execute(file_name):
    try:
        state = State()
        lines = svg_to_lines(file_name)
    except Exception as e:
        print('Error opening file \'', file_name, '\'', 'due to ', e)
        raise e
    
    if len(lines) <= 0:
        print('lines not found in file')
        return
    
    if lines[0][0] != 0 and lines[0][1] != 0:
        state = go_to(state, (0,0,lines[0][0],lines[0][1]))
        print('Moving to starting point (',lines[0][0],',',lines[0][1],') from origin')
    
    for line in lines:
        #print(line)
        if len(line) == 0:
            pass
        else:
            state = go_to(state, line)
            print('moving from (',line[0],',',line[1],') to (',line[2],',',line[3],')')
        
execute('2.png_SVG1.svg')

Moving to starting point ( 772 , 540 ) from origin
moving from ( 772 , 540 ) to ( 495 , 20 )
moving from ( 495 , 20 ) to ( 542 , 28 )
moving from ( 542 , 28 ) to ( 554 , 36 )
moving from ( 554 , 36 ) to ( 571 , 52 )
moving from ( 571 , 52 ) to ( 579 , 64 )
moving from ( 579 , 64 ) to ( 587 , 84 )
moving from ( 587 , 84 ) to ( 578 , 155 )
moving from ( 578 , 155 ) to ( 561 , 177 )
moving from ( 561 , 177 ) to ( 645 , 185 )
moving from ( 645 , 185 ) to ( 670 , 206 )
moving from ( 670 , 206 ) to ( 679 , 219 )
moving from ( 679 , 219 ) to ( 687 , 239 )
moving from ( 687 , 239 ) to ( 679 , 304 )
moving from ( 679 , 304 ) to ( 670 , 317 )
moving from ( 670 , 317 ) to ( 654 , 334 )
moving from ( 654 , 334 ) to ( 642 , 342 )
moving from ( 642 , 342 ) to ( 622 , 350 )
moving from ( 622 , 350 ) to ( 550 , 342 )
moving from ( 550 , 342 ) to ( 538 , 334 )
moving from ( 538 , 334 ) to ( 521 , 318 )
moving from ( 521 , 318 ) to ( 513 , 306 )
moving from ( 513 , 306 ) to ( 505 , 287 )
moving from ( 5

In [24]:
# This is here so I can compare it to what I'm getting up top
soup = bs(open('2.png_SVG1.svg'), 'xml')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<svg baseProfile="full" height="541" version="1.1" width="773" xmlns="http://www.w3.org/2000/svg" xmlns:ev="http://www.w3.org/2001/xml-events" xmlns:xlink="http://www.w3.org/1999/xlink">
 <defs/>
 <g fill="none" id="shapes">
  <line stroke="rgb(10%,10%,16%)" x1="772" x2="495" y1="540" y2="20"/>
  <line stroke="rgb(10%,10%,16%)" x1="495" x2="542" y1="20" y2="28"/>
  <line stroke="rgb(10%,10%,16%)" x1="542" x2="554" y1="28" y2="36"/>
  <line stroke="rgb(10%,10%,16%)" x1="554" x2="571" y1="36" y2="52"/>
  <line stroke="rgb(10%,10%,16%)" x1="571" x2="579" y1="52" y2="64"/>
  <line stroke="rgb(10%,10%,16%)" x1="579" x2="587" y1="64" y2="84"/>
  <line stroke="rgb(10%,10%,16%)" x1="587" x2="578" y1="84" y2="155"/>
  <line stroke="rgb(10%,10%,16%)" x1="578" x2="561" y1="155" y2="177"/>
  <line stroke="rgb(10%,10%,16%)" x1="561" x2="645" y1="177" y2="185"/>
  <line stroke="rgb(10%,10%,16%)" x1="645" x2="670" y1="185" y2="206"/>
  <line stroke="rgb(10%,10%,

## Explore .svg More
#### paste this code into https://www.w3schools.com/graphics/tryit.asp?filename=trysvg_polygon3

```
<!DOCTYPE html>
<html>
<body>

<svg height="300pt" width="600pt"> 

     <!-- This is the example shape @(250,250) instead of (8114, 278) -->
	<g fill="pink">
      <path stroke="none" d="M250 250 c-9 -42 -10 -43 -48 -40 l-39 3 31 -35 c30 -35 30 -35 17 -85 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11 z"/>
  	</g> 
    
    <!-- These are the individual part making the shape -->
	<g fill="none">
      <path stroke="red" d="M 250 250
   		c-9 -42 -10 -43 -48 -40 " />
      <path stroke="black" d="M 202 210 l-39 3 31 -35" />
      <path stroke="blue" d="M 194 178 c30 -35 30 -35 17 -85" />
      <path stroke="green" d="M 211 93 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46" />
      <path stroke="yellow" d="M 302 73 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11" />
  	</g>  
    
     <!-- Mark relevant points -->
     
     <!-- Below we can observe how curveto and lineto frustratingly differ-->
     <!-- lineto moves relative to the previous point in the command-->
     
     <!-- curveto has two type of points, which I'll call anchors and control -->
     <!-- the first and last points in curve to are anchors -->
     <!-- controls moves relative to the most recent anchor-->
     <!-- there seem to alway be two -->
     <!-- then there is another anchor, which also moves relative to the last anchor-->
     
     
    <g fill="red"> <!-- red curveto points -->
      <circle id="A" cx="250" cy="250" r="2" />
      <!-- NOTE: These are control points -->
      <circle fill="white" stroke="red" id="B" cx="241" cy="209" r="2" />
      <circle fill="white" stroke="red" id="C" cx="240" cy="207" r="2" />
      <circle id="D" cx="202" cy="210" r="4" />
    </g>
    <g fill="Black"> <!-- black lineto points -->
      <circle id="A" cx="202" cy="210" r="2" />
      <circle id="B" cx="163" cy="213" r="2" />
      <circle id="C" cx="194" cy="178" r="4" />
    </g>
    <g fill="Blue"> <!-- blue curve to points -->
      <circle id="A" cx="194" cy="178" r="2" />
      <!-- NOTE: These are control points at the same location-->
      <circle fill="white" stroke="blue" id="B" cx="224" cy="143" r="2" />
      <circle fill="white" stroke="blue" id="C" cx="224" cy="143" r="2" />
      <circle id="D" cx="211" cy="93" r="4" />
    </g>
    <g fill="green"> <!-- green lineto points -->
      <circle id="A" cx="211" cy="93" r="2" />
      <circle id="B" cx="198" cy="43" r="2" /> <!---13 -50-->
      <circle id="C" cx="228" cy="65" r="2" /> <!--30 22-->
      <circle id="D" cx="258" cy="87" r="2" /> <!-- 30 22 -->
      <circle id="E" cx="283" cy="57" r="2" /> <!--25 -30 --> 
      <circle id="F" cx="308" cy="27" r="2" /> <!-- 25 -30 --> 
      <circle id="G" cx="302" cy="73" r="4" /> <!-- -6 46 -->
    </g>
    <g fill="Yellow"> <!-- yellow curve to points -->
      <circle id="A" cx="302" cy="73" r="2" />
      <!--  Two control points -->
      <circle fill="black" stroke="yellow" id="B" cx="298" cy="113" r="2" />
      <circle fill="black" stroke="yellow" id="C" cx="300" cy="122" r="2" />
      <!-- new anchor -->
      <circle fill="yellow" id="D" cx="321" cy="142" r="2" />
      <!--  Two more control points -->
      <circle fill="black" stroke="yellow" id="E" cx="353" cy="172" r="2" />
      <circle fill="black" stroke="yellow" id="F" cx="352" cy="176" r="2" />
      <!-- new anchor -->
      <circle fill="yellow" id="D" cx="314" cy="187" r="2" />
      <!--  Two more control points -->
      <circle fill="black" stroke="yellow" id="E" cx="285" cy="198" r="2" />
      <circle fill="black" stroke="yellow" id="F" cx="280" cy="202" r="2" />
      <!-- new anchor -->
      <circle fill="yellow" id="D" cx="275" cy="239" r="2" />
      <!--  Two more control points -->
      <circle fill="black" stroke="yellow" id="E" cx="268" cy="297" r="2" />
      <circle fill="black" stroke="yellow" id="F" cx="261" cy="300" r="2" />
      <!--  Final Anchor / End Point / Closes loop -->
      <circle id="x" cx="250" cy="250" r="4" />
      
      <!-- -7 58 -14 61 -25 11 -->
    </g>
   Sorry, your browser does not support inline SVG.
  </svg>

</body>
</html>
```